In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

import copy
import random
import time


In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
train_transforms = transforms.Compose([
                            transforms.Grayscale(num_output_channels=1),
                            transforms.Resize(10),
                            transforms.ToTensor()
                                      ])
train_data = datasets.ImageFolder(root = 'dga_im',transform=train_transforms)

In [4]:
train_iterator = data.DataLoader(train_data, 
                                 shuffle = True, 
                                 batch_size = 8)

In [5]:
mean = 0.0
for images, _ in train_iterator:
    batch_samples = images.size(0) 
    images = images.view(batch_samples, images.size(1), -1)
    mean += images.mean(2).sum(0)
mean = mean / len(train_iterator.dataset)

var = 0.0
for images, _ in train_iterator:
    batch_samples = images.size(0)
    images = images.view(batch_samples, images.size(1), -1)
    var += ((images - mean.unsqueeze(1))**2).sum([0,2])
std = torch.sqrt(var / (len(train_iterator.dataset)*10*10))

In [6]:
print(mean)
print(std)

tensor([0.1446])
tensor([0.1971])


In [7]:
train_transforms = transforms.Compose([
                            transforms.Grayscale(num_output_channels=1),
                            transforms.Resize(10),
                            transforms.ToTensor(),
                            transforms.Normalize(mean = [mean], std = [std])
                                      ])
train_data = datasets.ImageFolder(root = 'dga_im',transform=train_transforms)
RATIO = 0.9
n_train_examples = int(len(train_data) * RATIO)
n_orginalvalid_examples = len(train_data) - n_train_examples
n_valid_examples = int((n_orginalvalid_examples)*0.5)
n_test_examples = n_orginalvalid_examples - n_valid_examples
print(n_train_examples)
print(n_valid_examples)
print(n_test_examples)

train_data, valid_data, test_data = data.random_split(train_data, 
                                           [n_train_examples, n_valid_examples, n_test_examples])

507
28
29


In [8]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')

Number of training examples: 507
Number of validation examples: 28


In [9]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')

Number of training examples: 507
Number of validation examples: 28


In [10]:
class LeNet(nn.Module):
    def __init__(self, output_dim):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels = 1, 
                               out_channels = 3, 
                               kernel_size = 3)
        
        self.conv2 = nn.Conv2d(in_channels = 3, 
                               out_channels = 6, 
                               kernel_size = 3)
        
        self.fc_1 = nn.Linear(216, 72)
        self.fc_2 = nn.Linear(72, 24)
        self.fc_3 = nn.Linear(24, output_dim)

    def forward(self, x):

        
        
        x = self.conv1(x)
        
        
        
        
        
        
        x = F.relu(x)
        
        x = self.conv2(x)
        
        

        
        
        
        x = F.relu(x)
        
        x = x.view(x.shape[0], -1)
        
        
        
        h = x
        
        x = self.fc_1(x)
        
        
        
        x = F.relu(x)

        x = self.fc_2(x)
        
        x = F.relu(x)

        x = self.fc_3(x)
    
    

        
        
        return x, h


In [11]:
OUTPUT_DIM = 6

model = LeNet(OUTPUT_DIM)

In [12]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 17,724 trainable parameters


In [13]:
optimizer = optim.Adam(model.parameters())

In [14]:
BATCH_SIZE=8
train_iterator = data.DataLoader(train_data, 
                                 shuffle = True, 
                                 batch_size = BATCH_SIZE)

valid_iterator = data.DataLoader(valid_data, 
                                 batch_size = BATCH_SIZE)

In [15]:
optimizer = optim.Adam(model.parameters())

In [16]:
criterion = nn.CrossEntropyLoss()

In [17]:
device = torch.device('cpu')

In [18]:
device

device(type='cpu')

In [19]:
model = model.to(device)
criterion = criterion.to(device)

In [20]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [21]:

def train(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for (x, y) in iterator:
        
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
                
        y_pred, _ = model(x)
        
        loss = criterion(y_pred, y)
        
        acc = calculate_accuracy(y_pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        
        for (x, y) in iterator:

            x = x.to(device)
            y = y.to(device)

            y_pred, _ = model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs





In [22]:
val_acc=[]
train_acc=[]

train_loss=[]
val_loss=[]
epoch_x_axis=[i for i in range(0,100)]

In [23]:
EPOCHS = 125
best_valid_loss = float('inf')

for epoch in range(EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut-model.pt')
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    


Epoch: 01 | Epoch Time: 0m 0s
	Train Loss: 1.738 | Train Acc: 24.74%
	 Val. Loss: 1.654 |  Val. Acc: 25.00%
Epoch: 02 | Epoch Time: 0m 0s
	Train Loss: 1.382 | Train Acc: 44.21%
	 Val. Loss: 1.289 |  Val. Acc: 46.88%
Epoch: 03 | Epoch Time: 0m 0s
	Train Loss: 1.181 | Train Acc: 52.93%
	 Val. Loss: 1.115 |  Val. Acc: 50.00%
Epoch: 04 | Epoch Time: 0m 0s
	Train Loss: 1.100 | Train Acc: 56.18%
	 Val. Loss: 1.031 |  Val. Acc: 50.00%
Epoch: 05 | Epoch Time: 0m 0s
	Train Loss: 1.030 | Train Acc: 60.35%
	 Val. Loss: 0.958 |  Val. Acc: 56.25%
Epoch: 06 | Epoch Time: 0m 0s
	Train Loss: 1.051 | Train Acc: 59.11%
	 Val. Loss: 0.943 |  Val. Acc: 50.00%
Epoch: 07 | Epoch Time: 0m 0s
	Train Loss: 0.991 | Train Acc: 61.07%
	 Val. Loss: 0.985 |  Val. Acc: 43.75%
Epoch: 08 | Epoch Time: 0m 0s
	Train Loss: 0.979 | Train Acc: 61.59%
	 Val. Loss: 0.889 |  Val. Acc: 59.38%
Epoch: 09 | Epoch Time: 0m 0s
	Train Loss: 0.982 | Train Acc: 62.17%
	 Val. Loss: 0.946 |  Val. Acc: 56.25%
Epoch: 10 | Epoch Time: 0m 0

In [24]:
test_transforms = transforms.Compose([
                            transforms.Grayscale(num_output_channels=1),
                            transforms.Resize(10),
                            transforms.ToTensor(),
                            transforms.Normalize(mean = [mean], std = [std])
                                      ])

test_iterator = data.DataLoader(test_data, 
                                batch_size = BATCH_SIZE)
model.load_state_dict(torch.load('tut-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.459 | Test Acc: 82.50%


In [25]:
dummy_input = torch.randn(8,1,10,10, device='cpu')

In [26]:
import torch.onnx
torch.onnx.export(model, dummy_input, "model_file.onnx",verbose=True)

graph(%input.1 : Float(8, 1, 10, 10),
      %conv1.weight : Float(3, 1, 3, 3),
      %conv1.bias : Float(3),
      %conv2.weight : Float(6, 3, 3, 3),
      %conv2.bias : Float(6),
      %fc_1.weight : Float(72, 216),
      %fc_1.bias : Float(72),
      %fc_2.weight : Float(24, 72),
      %fc_2.bias : Float(24),
      %fc_3.weight : Float(6, 24),
      %fc_3.bias : Float(6)):
  %11 : Float(8, 3, 8, 8) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[3, 3], pads=[0, 0, 0, 0], strides=[1, 1]](%input.1, %conv1.weight, %conv1.bias), scope: LeNet/Conv2d[conv1] # /opt/tljh/user/lib/python3.7/site-packages/torch/nn/modules/conv.py:340:0
  %12 : Float(8, 3, 8, 8) = onnx::Relu(%11), scope: LeNet # /opt/tljh/user/lib/python3.7/site-packages/torch/nn/functional.py:913:0
  %13 : Float(8, 6, 6, 6) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[3, 3], pads=[0, 0, 0, 0], strides=[1, 1]](%12, %conv2.weight, %conv2.bias), scope: LeNet/Conv2d[conv2] # /opt/tljh/user/lib/python3.7/site-package

In [27]:
import onnx
model=onnx.load_model('model_file.onnx')

In [28]:
import os
os.makedirs("./model_file_deepC", exist_ok=True)

In [29]:
!compile-onnx model_file.onnx ./model_file_deepC